In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
from collections import defaultdict
from tqdm import tqdm
from torch.utils.data import Subset
import numpy as np
import torch.nn.functional as F
from datasets import GoldCorrectionDataset
from glc import CorrectionGenerator, GoldCorrectionLossFunction

In [6]:
from torchvision.datasets import MNIST
trn_ds = MNIST('data/', train=True, transform=ToTensor())
val_ds = MNIST('data/', train=False, transform=ToTensor())

In [7]:
c_gen = CorrectionGenerator(dataset=trn_ds, randomization_strength=1.0)

100%|██████████| 6000/6000 [00:00<00:00, 6324.63it/s]


In [8]:
trusted_dataset, untrusted_dataset = c_gen.fetch_datasets()

In [12]:
from torched.customs.layers import LinearLayer, Flatten
from torched.trainer_utils import Train
class Net(nn.Module):
    def __init__(self, in_dims, hid_dims, out_dims):
        super(Net, self).__init__()
        self.net = nn.Sequential(Flatten(),
                                LinearLayer(in_dims, hid_dims, use_bn=True),
                                LinearLayer(hid_dims, hid_dims, use_bn=True),
                                nn.Linear(hid_dims, out_dims))
    def forward(self, x):
        if isinstance(x, list):
            inp, c = x[0], x[1]
        else:
            inp = x
        out = self.net(inp)
        if isinstance(x, list):
            return [out, c]
        return out

In [13]:
model = Net(784, 300, 10)
u_dl = DataLoader(untrusted_dataset, batch_size=32)
v_dl = DataLoader(val_ds, batch_size=32)
trainer = Train(model, [u_dl, v_dl], cuda=False)

In [14]:
trainer.train(1e-4, 3, 2, crit=nn.CrossEntropyLoss(), opt='adamW')

Valid Loss 2.348561: 100%|██████████| 313/313 [00:02<00:00, 112.15it/s]

In [15]:
label_correction_matrix = c_gen.generate_correction_matrix(trainer.model, 32)

Processing label 9: 100%|██████████| 22/22 [00:00<00:00, 125.26it/s]

Done


In [16]:
gold_ds = GoldCorrectionDataset(trusted_dataset, untrusted_dataset)
gold_dl = DataLoader(gold_ds, batch_size=32, shuffle=True)
g_val_ds = GoldCorrectionDataset(val_ds, val_ds, valid=True)
g_val_dl = DataLoader(g_val_ds, batch_size=32)
gold_loss = GoldCorrectionLossFunction(label_correction_matrix)

In [17]:
trainer.dataloader = gold_dl
trainer.val_dataloader = g_val_dl
trainer.train(1e-4, 3, 2, crit=gold_loss, opt='adamW')

Valid Loss 0.155766: 100%|██████████| 625/625 [00:06<00:00, 90.00it/s] 


In [18]:
def epoch(loader, model, opt=None):
    """Standard training/evaluation epoch over the dataset"""
    total_loss, total_err = 0.,0.
    for X,y in loader:
        X,y = X, y
        yp = model(X)
        loss = nn.CrossEntropyLoss()(yp,y)
        if opt:
            opt.zero_grad()
            loss.backward()
            opt.step()
        
        total_err += (yp.max(dim=1)[1] != y).sum().item()
        total_loss += loss.item() * X.shape[0]
    return total_err / len(loader.dataset), total_loss / len(loader.dataset)

In [19]:
err, loss = epoch(v_dl, trainer.model)

In [20]:
1 - err

0.9545